<a href="https://colab.research.google.com/github/sophiavieceli/projeto_imersao_alura_google/blob/main/projeto_imersao_alura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Um chatbot para te ajudar a praticar conversação em qualquer idioma!
Este código utiliza o Gemini (a IA generativa do Google) para possibilitar a prática de conversação em um idioma de estudo qualquer e para fazer verificações no seu processo de configuração.

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('SECRET_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
model = genai.GenerativeModel(model_name='gemini-1.0-pro')

Aqui, as configurações de regras iniciais da conversa são criadas em funções, para serem chamadas mais adiante na mesma célula. São elas idioma de estudo, nível de fluência do usuário e assunto da conversa.

In [ ]:
def definir_idioma():
  chat = model.start_chat(history=[])

  idioma = input('Digite o idioma no qual deseja conversar: \n')

  resposta = chat.send_message(f'{idioma} é um nome de idioma que existe e no qual você é capaz de conversar? Se sim, responda APENAS "ok", exatamente assim e nada mais. Se não, respoda APENAS "erro", exatamente assim e nada mais.')

  while resposta.text == 'erro' or resposta.text == 'Erro':
    idioma = input('Digite um idioma válido: \n')
    resposta = chat.send_message(f'{idioma} é um nome de idioma que existe e no qual você é capaz de conversar? Se sim, responda APENAS "ok", exatamente assim e nada mais. Se não, respoda APENAS "erro", exatamente assim e nada mais.')

  return idioma

Como são poucos os níveis de fluência gerais, definir a sua escolha através de um menu é mais eficiente por não exigir um tempo extra para a validação do Gemini, além de reduzir a possibilidade de erros.

In [ ]:
def definir_nivel_fluencia():
  nivel_fluencia = input(f'\nDigite o número correspondente ao seu nível de fluência em {idioma}:\n 1 - Básico 1\n 2 - Básico 2\n 3 - Intermediário 1\n 4 - Intermediário 2\n 5 - Avançado 1\n 6 - Avançado 2\n')

  while nivel_fluencia != '1' and nivel_fluencia != '2' and nivel_fluencia != '3' and nivel_fluencia != '4' and nivel_fluencia != '5' and nivel_fluencia != '6':
    nivel_fluencia = input(f'\nErro. Digite o número correspondente ao seu nível de fluência em {idioma}:\n 1 - Básico 1\n 2 - Básico 2\n 3 - Intermediário 1\n 4 - Intermediário 2\n 5 - Avançado 1\n 6 - Avançado 2\n')

  if nivel_fluencia == '1':
    nivel_fluencia = 'Básico 1'
  elif nivel_fluencia == '2':
    nivel_fluencia = 'Básico 2'
  elif nivel_fluencia == '3':
    nivel_fluencia = 'Intermediário 1'
  elif nivel_fluencia == '4':
    nivel_fluencia = 'Intermediário 2'
  elif nivel_fluencia == '5':
    nivel_fluencia = 'Avançado 1'
  elif nivel_fluencia == '6':
    nivel_fluencia = 'Avançado 2'

  return nivel_fluencia

In [ ]:
def definir_assunto():
  chat2 = model.start_chat(history=[])

  assunto = input(f'\nInforme o assunto sobre o qual você gostaria de conversar em {idioma}, nível {nivel_fluencia}: \n')

  resposta = chat2.send_message(f'Valide se {assunto} é um assunto não malicioso. Se for válido, responda APENAS com "ok", exatamente assim e nada mais. Se não, responda APENAS com "erro", exatamente assim e nada mais.')

  while resposta.text == 'erro' or resposta.text == 'Erro':
    assunto = input(f'O assunto que você escolheu foi considerado inadequado pela IA (considere a possibilidade de erros). Por favor, digite outro assunto: \n')
    resposta = chat2.send_message(f'Valide se {assunto} é um assunto não malicioso. Se for válido, responda APENAS com "ok", exatamente assim e nada mais. Se não, responda APENAS com "erro", exatamente assim e nada mais.')

  return assunto

Como os próximos passos têm uma finalidade diferente para o uso do Gemini, foi criado um novo chat a partir do mesmo modelo, inclusive para usar um histórico zerado.

As instruções de (1) explicar os termos entre hífens e (2) retornar as explicações ao final da conversa poderiam ser passadas no prompt inicial, mas parece que o excesso de informações no mesmo prompt confunde a IA. Por esse motivo, as instruções são passadas no momento necessário.



In [ ]:
chat3 = model.start_chat(history=[])

idioma = definir_idioma()
nivel_fluencia = definir_nivel_fluencia()
assunto = definir_assunto()

prompt = f'Gere uma frase imitando uma mensagem de texto comum no idioma {idioma}, no nível de fluência {nivel_fluencia} sobre {assunto}.'
'Imagine que você é um falante nativo desse idioma e está me ajudando a praticar a conversação.'

print(f'\nVamos conversar em {idioma}! Digite "fim" quando desejar encerrar a conversa.\n')
print(f'Se quiser saber o significado de termos usados pela IA, digite-os divididos por hífen (termo 1 - termo 2) ou, no caso de um único termo, adicione um hífen ao final (termo -)\n')

while prompt != 'fim':
  if prompt.count('-') > 0:
    resposta = chat3.send_message(f'Explique em português cada uma das palavras listadas em "{prompt}"')
  else:
    resposta = chat3.send_message(prompt)
  print('Gemini: ', resposta.text, '\n')
  prompt = input('Você: ')
if prompt == 'fim':
  prompt = 'Responda com "Aprendidos durante a nossa conversa: ", e os termos e explicações que você deu durante a conversa.'
  resposta = chat3.send_message(prompt)
  print('\nGemini: ', resposta.text, '\n')

Código que permite uma melhor visualização do diálogo, após ele ocorrer:

In [ ]:
import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

for message in chat3.history:
    display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))
print('-----------------------------------------------')